# **Vektorandmebaasid**

# Word2Vec mudel, et muuta tekst vektoriks.
Kasutame selleks **gensim** teeki, mis pakub tuge Word2Vec mudelile.

In [5]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import numpy as np

# Näiteks treenime väikese mudeli
sentences = [
    ['tere', 'kuidas', 'sul', 'läheb'],
    ['ma', 'olen', 'hea', 'eesti', 'keeles'],
    ['tänan', 'suure', 'abi', 'ja', 'toetuse','eest']
]

# Loome ja treenime mudeli
model = Word2Vec(sentences, vector_size=5, window=3, min_count=1, workers=4)

def text_to_vector(text, model):
    # Eelprotsessime teksti (nt. väikesed tähed ja jagame sõnadeks)
    tokens = simple_preprocess(text)
    vectors = []

    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])

    # Kui vektoreid on, arvutame keskmise; kui mitte, tagastame None
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return None

# Näide teksti konverteerimisest vektoriks
text = "Tere, kuidas sul läheb?"
vector = text_to_vector(text, model)

text2 = "Eesti keel on võõras"
vector2 = text_to_vector(text2, model)

text3 = "Eesti keel on hea"
vector3 = text_to_vector(text3, model)

if vector is not None:
    print("Lause vektor:", vector)
else:
    print("Tekstis ei leitud ühtegi tuntud sõna mudelist.")

print("Lause 2 vektor", vector2)
print("Lause 3 vektor", vector3)

Lause vektor: [ 0.0094288   0.06148399 -0.03423835 -0.11591148  0.07260559]
Lause 2 vektor [-0.03875482  0.16154873 -0.11861791  0.00090325 -0.09507468]
Lause 3 vektor [-0.11541291  0.1308473  -0.14690481 -0.04346663 -0.04788834]


# 1. Näide: Teksti muundamine vektoriks kasutades GloVe mudelit
**GloVe (Global Vectors for Word Representation)**
on teine populaarne sõnade vektorite mudel. Kasutame selle mudeli gensim versiooni, mille saab alla laadida ja kasutada.

In [6]:
from gensim.models import KeyedVectors
import numpy as np
import urllib.request
import zipfile

# Laadime GloVe mudeli alla ja ekstraktime selle
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
filename = 'glove.6B.zip'

# Laadime alla ja ekstrakteerime
urllib.request.urlretrieve(url, filename)
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

# Laadime mudeli
model_path = 'glove.6B.50d.txt'  # Kasutame 50-mõõtmelist mudelit
glove_model = KeyedVectors.load_word2vec_format(model_path, binary=False, no_header=True)

def text_to_vector_glove(text, model):
    tokens = text.lower().split()
    vectors = [model.get_vector(token) for token in tokens if token in model]

    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return None

# Näide teksti konverteerimisest vektoriks
text = "Tere, kuidas sul läheb?"
vector = text_to_vector_glove(text, glove_model)

if vector is not None:
    print("Lause vektor:", vector)
else:
    print("Tekstis ei leitud ühtegi tuntud sõna mudelist.")


Lause vektor: [ 0.15939  -0.30956   0.12754   0.76091  -1.5251   -1.1138   -0.40551
 -0.041995  0.15534   0.92986   0.55406  -0.28308   0.35449  -0.79131
 -0.028066  0.33871   0.66246   0.14486   0.52719   1.4927   -1.2364
 -1.1385    1.029     0.92151  -0.80055  -0.92212  -0.89299   0.43219
 -1.2797   -1.0169   -0.16437   0.051815 -0.43075   0.87365   0.81784
 -1.1261   -0.94363   0.54744   1.4867    0.98699  -1.0787    0.96754
  0.42122  -0.76563  -1.2348   -0.1377    0.34152  -0.17419   0.30207
 -0.33281 ]


# 2. Näide: Teksti muundamine vektoriks kasutades BERT mudelit
**BERT (Bidirectional Encoder Representations from Transformers)**
on kaasaegne mudel, mis genereerib sõnade ja lausete vektoreid arvestades konteksti. Kasutame transformers teeki.

In [7]:
from transformers import BertTokenizer, BertModel
import torch

# Laadime BERT mudeli ja tokenizeri
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def text_to_vector_bert(text, tokenizer, model):
    # Tokeniseerime teksti
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)

    # Viimane peidetud kiht
    last_hidden_states = outputs.last_hidden_state

    # Arvutame keskmise peidetud oleku
    sentence_vector = last_hidden_states.mean(dim=1).squeeze()
    return sentence_vector.numpy()

# Näide teksti konverteerimisest vektoriks
text = "Tere, kuidas sul läheb?"
vector = text_to_vector_bert(text, tokenizer, model)

print("Lause vektor:", vector)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Lause vektor: [-2.01246262e-01  1.91958819e-03 -5.53269029e-01 -1.66291416e-01
  1.67682007e-01  2.07082897e-01  3.89076412e-01  4.37816203e-01
  1.42115459e-01 -1.73542947e-01  7.72437304e-02 -4.01706159e-01
 -2.47971922e-01  6.25092149e-01 -3.62704545e-01  8.30090761e-01
  8.81022140e-02 -2.31596634e-01  7.34257698e-02  2.41620049e-01
 -7.66636338e-03 -7.23251849e-02 -1.23451382e-01  9.54457931e-03
 -1.78358167e-01  2.92896539e-01  1.15153663e-01  3.07832211e-01
 -3.81147385e-01  1.12593919e-01  6.45089447e-02  9.78155527e-03
 -6.51681244e-01 -1.74371302e-01 -3.60636294e-01  3.85353774e-01
  1.84578970e-01  3.27835861e-03  6.41734749e-02 -8.23216438e-02
 -5.21866560e-01 -4.93340850e-01 -3.34130794e-01  4.27582532e-01
  5.64362630e-02 -3.86333197e-01  6.50619641e-02 -2.83067971e-01
 -3.88346553e-01 -1.89941511e-01 -6.49860919e-01  1.75131634e-01
 -7.28281066e-02 -4.96762432e-02 -3.57390404e-01  6.86089788e-03
 -5.40781254e-03 -2.98580587e-01  2.54738539e-01 -3.01497057e-02
 -4.9024485

# 3. Näide: Tekstide sarnastekuse hindamine
Kasutame **Word2Vec** mudelit, et hinnata, kui sarnased on kaks teksti lauset, võrreldes nende keskmiste vektorite sarnasust.

In [11]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from scipy.spatial.distance import cosine
import numpy as np

# Näiteks treenime väikese mudeli
sentences = [
    ['tere', 'kuidas', 'sul', 'läheb'],
    ['ma', 'olen', 'hea', 'eesti', 'keeles'],
    ['tänan', 'suure', 'abi', 'ja', 'toetuse', 'eest'],
    ['tere', 'kuidas', 'läheb'],
    ['mis', 'kell', 'on']
]

model = Word2Vec(sentences, vector_size=50, window=3, min_count=1, workers=4)

def text_to_vector(text, model):
    tokens = simple_preprocess(text)
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return None

def cosine_similarity(vector1, vector2):
    return 1 - cosine(vector1, vector2)

text1 = "Tere, kuidas sul läheb?"
text2 = "Kuidas sul läheb?"
text3 = "Mis kell on?"

vector1 = text_to_vector(text1, model)
vector2 = text_to_vector(text2, model)
vector3 = text_to_vector(text3, model)

if vector1 is not None and vector2 is not None:
    similarity = cosine_similarity(vector1, vector2)
    print(f"Tekstide sarnasus: {similarity:.4f}")
else:
    print("Üks või mõlemad tekstid ei ole mudelis.")


if vector2 is not None and vector3 is not None:
    similarity = cosine_similarity(vector2, vector3)
    print(f"Tekstide sarnasus: {similarity:.4f}")
else:
    print("Üks või mõlemad tekstid ei ole mudelis.")

Tekstide sarnasus: 0.8726
Tekstide sarnasus: -0.0597


# 4. Näide: Sõnade määratlemine semantilise sarnasuse järgi
Selles näites arvutame, kui sarnased on kaks sõna, kasutades **GloVe** mudeli vektoreid.

In [31]:
from gensim.models import KeyedVectors
import numpy as np
import urllib.request
import zipfile
from scipy.spatial.distance import cosine

# Laadime GloVe mudeli alla ja pakime lahti selle
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
filename = 'glove.6B.zip'
'''
urllib.request.urlretrieve(url, filename)
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()'''

# Laadime mudeli
model_path = 'glove.6B.50d.txt'  # Kasutame 50-mõõtmelist mudelit
glove_model = KeyedVectors.load_word2vec_format(model_path, binary=False, no_header=True)

def word_similarity(word1, word2, model):
    if word1 in model and word2 in model:
        vector1 = model[word1]
        vector2 = model[word2]
        return 1 - cosine(vector1, vector2)
    else:
        return None

word1 = "king"
word2 = "shoe"

word1 = "king"
word2 = "queen"

similarity = word_similarity(word1, word2, glove_model)

if similarity is not None:
    print(f"Sõnade '{word1}' ja '{word2}' sarnasus: {similarity:.4f}")
else:
    print("Üks või mõlemad sõnad ei ole mudelis.")


Sõnade 'king' ja 'queen' sarnasus: 0.7839


# 5. Näide: Teksti klassifitseerimine
Tekstide vektorid saavad sisendiks klassifikaatorile.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import numpy as np

# Näiteks treenime väikese mudeli
train_texts = [
    'Tere, kuidas sul läheb?',
    'Ma armastan koeri.',
    'Kui tore päev!',
    'Kuidas saavad koerad ja kassid omavahel läbi?',
    'Täna on ilm väga halb.'
]

train_labels = ['tervitus', 'lemmikloomad', 'meeleolu', 'lemmikloomad', 'ilm']

# Loome lihtsa tekstiklassifikaatori
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Treenime mudelit
model.fit(train_texts, train_labels)

# Näide teksti klassifitseerimisest
test_texts = [
    'Mis on ilm täna?',
    'Kuidas koerad tunnevad end ilma?',
]

predictions = model.predict(test_texts)

for text, prediction in zip(test_texts, predictions):
    print(f"Tekst: '{text}'\nKlass: {prediction}\n")


Tekst: 'Mis on ilm täna?'
Klass: ilm

Tekst: 'Kuidas koerad tunnevad end ilma?'
Klass: lemmikloomad



# 6. Näide: Teksti klasterdamine vektorite põhjal
Siin kasutame **KMeans algoritmi**, et klasterdada tekste nende vektorite põhjal. Kasutame **Word2Vec mudelit**, et luua tekstivektorid, ning seejärel jagame tekstid klastritesse.

In [15]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import numpy as np

# Näiteks treenime väikese mudeli
sentences = [
    ['tere', 'kuidas', 'sul', 'läheb'],
    ['ma', 'olen', 'hea', 'eesti', 'keeles'],
    ['tänan', 'suurt', 'abil', 'ja', 'toetuse'],
    ['kassid', 'ja', 'koerad', 'on', 'lemmikloomad'],
    ['ilm', 'on', 'täna', 'väga', 'halb'],
    ['tahaksin', 'koeri', 'ja', 'kasse', 'armastada']
]

model = Word2Vec(sentences, vector_size=50, window=3, min_count=1, workers=4)

def text_to_vector(text, model):
    tokens = text.lower().split()
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return None

# Tekstide klasterdamine
texts = [
    "Tere, kuidas sul läheb?",
    "Kassid on lemmikloomad.",
    "Täna on ilm halb.",
    "Koerad on väga sõbralikud.",
    "Ilm on kohutav!",
    "Ma armastan kasse."
]

vectors = [text_to_vector(text, model) for text in texts]
vectors = np.array([vec for vec in vectors if vec is not None])

# Klasterdamine KMeans algoritmi abil
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(vectors)

# Tekstide klastrite kuvamine
for i, text in enumerate(texts):
    print(f"Tekst: '{text}' kuulub klastri {kmeans.labels_[i]} alla")


Tekst: 'Tere, kuidas sul läheb?' kuulub klastri 0 alla
Tekst: 'Kassid on lemmikloomad.' kuulub klastri 0 alla
Tekst: 'Täna on ilm halb.' kuulub klastri 0 alla
Tekst: 'Koerad on väga sõbralikud.' kuulub klastri 0 alla
Tekst: 'Ilm on kohutav!' kuulub klastri 0 alla
Tekst: 'Ma armastan kasse.' kuulub klastri 1 alla


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# Dokumendi sarnaste tekstide otsimine andmebaasist
Siin näites loome süsteemi, mis võimaldab otsida andmebaasist sarnaseid tekste. Kasutame BERT mudelit, et luua iga dokumendi vektor ja seejärel otsime kõige sarnasemaid dokumente.

In [16]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

# Laadime BERT mudeli ja tokenizeri
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def text_to_vector_bert(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    sentence_vector = last_hidden_states.mean(dim=1).squeeze()
    return sentence_vector.numpy()

# Näiteks andmebaasi dokumendid
documents = [
    "Koerad on väga sõbralikud.",
    "Täna on ilm halb ja vihmane.",
    "Ma armastan kasse ja koeri.",
    "Kassid on ilusad lemmikloomad.",
    "Ilm on täna tõesti halb.",
]

# Dokumendi vektorid
doc_vectors = np.array([text_to_vector_bert(doc, tokenizer, model) for doc in documents])

# Otsitav tekst
query_text = "Milline on ilm täna?"
query_vector = text_to_vector_bert(query_text, tokenizer, model)

# Sarnasuste arvutamine
similarities = cosine_similarity([query_vector], doc_vectors)[0]

# Kuvame kõige sarnasemad dokumendid
most_similar_idx = np.argsort(similarities)[::-1]
print(f"Otsitav tekst: '{query_text}'")
for idx in most_similar_idx:
    print(f"Sarnane dokument: '{documents[idx]}' sarnasusega: {similarities[idx]:.4f}")


Otsitav tekst: 'Milline on ilm täna?'
Sarnane dokument: 'Ilm on täna tõesti halb.' sarnasusega: 0.9087
Sarnane dokument: 'Täna on ilm halb ja vihmane.' sarnasusega: 0.8670
Sarnane dokument: 'Koerad on väga sõbralikud.' sarnasusega: 0.8378
Sarnane dokument: 'Ma armastan kasse ja koeri.' sarnasusega: 0.8291
Sarnane dokument: 'Kassid on ilusad lemmikloomad.' sarnasusega: 0.7752


# Sõnade asenduste leidmine vektoriruumi abil
Siin näites kasutame Word2Vec mudelit, et leida sõnale semantiliselt sarnaseid sõnu, mida saaks tekstis kasutada asendusena.

In [22]:
from gensim.models import Word2Vec

# Näiteks treenime väikese mudeli
sentences = [
    ['tere', 'kuidas', 'sul', 'läheb'],
    ['ma', 'olen', 'hea', 'eesti', 'keeles'],
    ['tänan', 'suurt', 'abil', 'ja', 'toetuse'],
    ['koerad', 'ja', 'kassid', 'on', 'lemmikloomad'],
    ['ilm', 'on', 'täna', 'väga', 'halb']
]

model = Word2Vec(sentences, vector_size=5, window=3, min_count=1, workers=4)

# Sõna sarnaste sõnade leidmine
word = "väga"
if word in model.wv:
    similar_words = model.wv.most_similar(word, topn=5)
    print(f"Sõnale '{word}' sarnased sõnad:")
    for similar_word, similarity in similar_words:
        print(f"'{similar_word}' sarnasusega: {similarity:.4f}")
else:
    print(f"Sõna '{word}' ei leitud mudelist.")


Sõnale 'väga' sarnased sõnad:
'kuidas' sarnasusega: 0.6934
'lemmikloomad' sarnasusega: 0.6027
'hea' sarnasusega: 0.5808
'läheb' sarnasusega: 0.4890
'sul' sarnasusega: 0.4695


# Teksti klasterdamine
Kuna Word2Vec mudel treeniti väga väikese andmekogumi peal, ei pruugi teksti klasterdamine anda head tulemust. Siin on mõned parandusettepanekud:

Kasutage eel-treenitud mudelit: Selle asemel, et treenida Word2Vec mudelit väikese andmestikuga, võite kasutada eel-treenitud mudelit, näiteks Google'i eel-treenitud Word2Vec mudelit.

Treenige suurema andmestikuga: Kui treenite mudelit oma andmetega, siis kasutage suuremat ja mitmekesisemat korpust.

Kasutame eel-treenitud GloVe mudelit:

In [29]:
from gensim.models import KeyedVectors
from sklearn.cluster import KMeans
import numpy as np

# Laadime eel-treenitud GloVe mudeli
model_path = 'glove.6B.50d.txt'
glove_model = KeyedVectors.load_word2vec_format(model_path, binary=False, no_header=True)

def text_to_vector_glove(text, model):
    tokens = text.lower().split()
    vectors = [model.get_vector(token) for token in tokens if token in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return None

# Tekstide klasterdamine
texts = [
    "Tere, kuidas sul läheb?",
    "Kassid on lemmikloomad.",
    "Täna on ilm halb.",
    "Koerad on väga sõbralikud.",
    "Ilm on kohutav!",
    "Ma armastan kasse."
]

vectors = [text_to_vector_glove(text, glove_model) for text in texts]
vectors = np.array([vec for vec in vectors if vec is not None])

# Klasterdamine KMeans algoritmi abil
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(vectors)

# Tekstide klastrite kuvamine
for i, text in enumerate(texts):
    print(f"Tekst: '{text}' kuulub klastri {kmeans.labels_[i]} alla")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Tekst: 'Tere, kuidas sul läheb?' kuulub klastri 2 alla
Tekst: 'Kassid on lemmikloomad.' kuulub klastri 1 alla
Tekst: 'Täna on ilm halb.' kuulub klastri 1 alla
Tekst: 'Koerad on väga sõbralikud.' kuulub klastri 1 alla
Tekst: 'Ilm on kohutav!' kuulub klastri 1 alla
Tekst: 'Ma armastan kasse.' kuulub klastri 0 alla


# Dokumendi sarnaste tekstide otsimine andmebaasist
Mudeli suurus: BERT võib olla ressursimahukas ja mõnikord väiksema distilBERT mudeli kasutamine võib anda kiiremaid tulemusi ilma märgatava täpsuse vähenemiseta.

Teksti eelprotsessimine: Teksti eelprotsessimine võib samuti mõjutada tulemusi.

In [24]:
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

# Laadime DistilBERT mudeli ja tokenizeri
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def text_to_vector_bert(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    sentence_vector = last_hidden_states.mean(dim=1).squeeze()
    return sentence_vector.numpy()

# Näiteks andmebaasi dokumendid
documents = [
    "Koerad on väga sõbralikud.",
    "Täna on ilm halb ja vihmane.",
    "Ma armastan kasse ja koeri.",
    "Kassid on ilusad lemmikloomad.",
    "Ilm on täna tõesti halb.",
]

# Dokumendi vektorid
doc_vectors = np.array([text_to_vector_bert(doc, tokenizer, model) for doc in documents])

# Otsitav tekst
query_text = "Milline on ilm täna?"
query_vector = text_to_vector_bert(query_text, tokenizer, model)

# Sarnasuste arvutamine
similarities = cosine_similarity([query_vector], doc_vectors)[0]

# Kuvame kõige sarnasemad dokumendid
most_similar_idx = np.argsort(similarities)[::-1]
print(f"Otsitav tekst: '{query_text}'")
for idx in most_similar_idx:
    print(f"Sarnane dokument: '{documents[idx]}' sarnasusega: {similarities[idx]:.4f}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Otsitav tekst: 'Milline on ilm täna?'
Sarnane dokument: 'Ilm on täna tõesti halb.' sarnasusega: 0.9110
Sarnane dokument: 'Täna on ilm halb ja vihmane.' sarnasusega: 0.8862
Sarnane dokument: 'Ma armastan kasse ja koeri.' sarnasusega: 0.8402
Sarnane dokument: 'Koerad on väga sõbralikud.' sarnasusega: 0.8165
Sarnane dokument: 'Kassid on ilusad lemmikloomad.' sarnasusega: 0.7883


# Sõnade asenduste leidmine vektoriruumi abil
Word2Vec mudel võib anda häid tulemusi, kui see on hästi treenitud.
GoogleNews-vectors-negative300.bin on suur fail (~1.5 GB), mida tuleb eraldi alla laadida ja seejärel kasutada.

In [26]:
from gensim.models import KeyedVectors

# Laadime eel-treenitud Google Word2Vec mudeli
model_path = 'GoogleNews-vectors-negative300.bin'
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

# Sõna sarnaste sõnade leidmine
word = "man"
if word in word2vec_model:
    similar_words = word2vec_model.most_similar(word, topn=5)
    print(f"Sõnale '{word}' sarnased sõnad:")
    for similar_word, similarity in similar_words:
        print(f"'{similar_word}' sarnasusega: {similarity:.4f}")
else:
    print(f"Sõna '{word}' ei leitud mudelist.")


FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

# Kasutame eel-treenitud GloVe mudelit
Võime kasutada eel-treenitud GloVe mudelit, mis on saadaval erinevates mõõtmetes. Saame laadida 50-mõõtmelise mudeli, mis on piisavalt väike ja kergesti kättesaadav.

https://nlp.stanford.edu/data/glove.6B.zip

In [27]:
import numpy as np

# Laadime GloVe mudeli failist
def load_glove_model(glove_file):
    print("Loading GloVe Model...")
    model = {}
    with open(glove_file, 'r', encoding="utf-8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            model[word] = embedding
    print(f"GloVe Model loaded with {len(model)} words.")
    return model

glove_file = 'glove.6B.50d.txt'  # Asenda õige failitee järgi
glove_model = load_glove_model(glove_file)

# Sõna sarnaste sõnade leidmine
def find_similar_words(word, model, topn=5):
    if word not in model:
        print(f"Sõna '{word}' ei leitud mudelist.")
        return

    word_vector = model[word]
    similarities = {}

    for other_word, other_vector in model.items():
        similarity = np.dot(word_vector, other_vector) / (np.linalg.norm(word_vector) * np.linalg.norm(other_vector))
        similarities[other_word] = similarity

    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)

    print(f"Sõnale '{word}' sarnased sõnad:")
    for similar_word, similarity in sorted_similarities[1:topn+1]:
        print(f"'{similar_word}' sarnasusega: {similarity:.4f}")

# Näide kasutamisest
find_similar_words("dog", glove_model)


Loading GloVe Model...
GloVe Model loaded with 400000 words.
Sõnale 'dog' sarnased sõnad:
'cat' sarnasusega: 0.9218
'dogs' sarnasusega: 0.8513
'horse' sarnasusega: 0.7908
'puppy' sarnasusega: 0.7755
'pet' sarnasusega: 0.7725
